In [1]:
import json
import requests
import sys
import time
sys.path.append('../ingestion_scripts/')
from ingestion import *

In [86]:
merged_keys = [
    "Tool_Name",
    "Tool_Description",
    "Institution",
    "Homepage",
    "$validator"
]

convert = [
    "Added_On",
    "Published_On",
    "Last_Updated"
]

merge_this = [
    "Keyword_List",
    "Chemical_List"
]


added = [
    "Social_Media_Mentions",
    "Tweets",
    "Facebook_Shares",
    "News_sources_that_mentioned_the_publication",
    "Readers_Count",
    "Citations",
    "Readers_Count",
    "Readers_in_Connotea",
    "Readers_in_Mendeley",
    "Cited_By_Posts_Count",
] 

throwaway = [
    "Icon_link",
    "Publicaions_in_this_journal",
    "Users_who've_mentioned_the_publication_on_Twitter",
    "Scientists_who_mentioned__the_publication_on_Twitter"
]

renamed_map = {
    "Mentions_in_social_media" : "Social_Media_Mentions",
    "Twitter_accounts_that_tweeted_this_publication": "Tweets",
    "tool_homepage_url": "Homepage",
    "KeywordList": "Keyword_List"
}


In [87]:
def renamed(k):
    if k in renamed_map:
        return (renamed_map[k])
    else:
        return k


def fetch_list(obj, key):
    try:
        return obj[key]
    except Exception as identifier:
        return []

In [88]:

with open("data/signatures_cfde_unmerged_021920.json") as o:
    tools = json.loads(o.read())

In [89]:
tool_homepage = {}
for t in tools:
    homepage = t["meta"]["tool_homepage_url"]
    tool_meta = {
        "id": t["id"],
        "library": t["library"],
        "meta": {
            "Publications": []
        }
    }
    pub = {}
    for k,v in t["meta"].items():
        key = renamed(k)
        if key not in throwaway:
            if key in convert:
                if v:
                    pub[key] = time.strftime('%Y-%m-%d', time.localtime(v))
                    if key == "Published_On":
                        tool_meta["meta"]["Year_First_Published"] = time.strftime('%Y', time.localtime(v))
            elif key == "Article_Date" and "Published_On" not in pub and len(v) > 0:
                pub["Published_On"] = "%s-%s-%s"%(v[0]["Year"],v[0]["Month"],v[0]["Day"])
                tool_meta["meta"]["Year_First_Published"] = v[0]["Year"]
            elif key in merged_keys:
                tool_meta["meta"][key] = v
            elif key in merge_this:
                tool_meta["meta"][key] = v
            elif key in added:
                pub[key] = v
                tool_meta["meta"][key] = v
            else:
                pub[key] = v
    if homepage not in tool_homepage:
        tool_meta["meta"]["Publications"].append(pub)
        tool_homepage[homepage] = tool_meta
    else:
        existing  = tool_homepage[homepage]
        existing_published = existing["meta"].get("Year_First_Published")
        curr_published = tool_meta["meta"].get("Year_First_Published")
        if existing_published and int(existing_published) <= int(curr_published): # The one in the dict is published first
            # append publication
            existing["meta"]["Publications"].append(pub)
            # add stuff
            for key in added:
                if key in tool_meta["meta"] and key in existing["meta"]:
                    existing["meta"][key] += tool_meta["meta"][key]
                elif key in tool_meta["meta"]:
                    existing["meta"][key] = tool_meta["meta"][key]        
            # Merge list
            existing_keywords = set(fetch_list(existing["meta"],"Keyword_List"))
            curr_keywords = set(fetch_list(tool_meta["meta"],"Keyword_List"))
            merged_list = list(existing_keywords.union(curr_keywords))
            if len(merged_list) > 0:
                existing["meta"]["Keyword_List"] = list(merged_list)
            existing_chem = fetch_list(existing["meta"],"Chemical_List")
            curr_chem = fetch_list(tool_meta["meta"],"Chemical_List")
            chem = {}
            for v in existing_chem:
                chem[v["NameOfSubstance"]] = v
            for v in curr_chem:
                chem[v["NameOfSubstance"]] = v
            existing["meta"]["Chemical_List"] = list(chem.values())
            # update existing       
            tool_homepage[homepage] = existing
        else:
            # append publication
            tool_meta["meta"]["Publications"].append(pub)
            # add stuff
            for key in added:
                if key in tool_meta["meta"] and key in existing["meta"]:
                    tool_meta["meta"][key] += existing["meta"][key]
                elif key in existing["meta"]:
                    tool_meta["meta"][key] = existing["meta"][key]
            # Merge list
            tool_meta_keywords = set(fetch_list(tool_meta["meta"],"Keyword_List"))
            existing_keywords = set(fetch_list(existing["meta"],"Keyword_List"))
            merged_list = list(tool_meta_keywords.union(existing_keywords))
            if len(merged_list) > 0:
                tool_meta["meta"]["Keyword_List"] = list(merged_list)
            tool_meta_chem = set(fetch_list(tool_meta["meta"],"Chemical_List"))
            existing_chem = set(fetch_list(existing["meta"],"Chemical_List"))
            chem = {}
            for v in tool_meta_chem:
                chem[v["NameOfSubstance"]] = v
            for v in existing_chem:
                chem[v["NameOfSubstance"]] = v
            tool_meta["meta"]["Chemical_List"] = list(chem.values())
            # update tool_meta       
            tool_homepage[homepage] = tool_meta


In [90]:
len(tool_homepage)

261

In [91]:
len(tools)

266

In [95]:
def empty_cleaner(obj):
    if type(obj) == str:
        obj = obj.strip()
        if obj == "":
            return None
        else:
            return obj
    elif type(obj) == list:
        new_list = []
        for i in obj:
            v = empty_cleaner(i)
            if v:
                new_list.append(v)
        if len(new_list) > 0:
            return new_list
        else:
            return None
    elif type(obj) == dict:
        new_dict = {}
        for k,v in obj.items():
            val = empty_cleaner(v)
            if val:
                new_dict[k] = val
        if len(new_dict) > 0:
            return new_dict
        else:
            return None
    else:
        return obj

In [96]:
tools_cleaned = list(tool_homepage.values())
tools_empty_cleaned = empty_cleaner(tools_cleaned)

In [97]:
with open("tool_cleaned_022020.json", "w") as o:
    o.write(json.dumps(tools_empty_cleaned)) 

In [74]:
tools_empty_cleaned